In [1]:
import os
import xml.etree.ElementTree as et

In [2]:
# Extracts all sitemap path to a list for preview

SLTrader_sitemap_path = "C:/Projects/StockLoans/Src/SLTrader/SiteMaps"

SLTrader_sitemap_lst = []

for root, dirs, files in os.walk(SLTrader_sitemap_path):
    for file in files:
        with open(os.path.join(root, file), 'r') as f:
            my_dir = str(os.path.join(root,file))
            target_dir = my_dir.replace('\\', '/')
            SLTrader_sitemap_lst.append(target_dir)
            
SLTrader_sitemap_lst

['C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-Administraton.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-CashSourcing.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-Compliance.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-Dashboard.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-DomesticTrading.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-FailMaster.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-Header.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-Locate.sitemap',
 'C:/Projects/StockLoans/Src/SLTrader/SiteMaps/Web-RebateBilling.sitemap']

In [3]:
# Parses xml sitemap to a nested dict

def parse_xml_sitemap(path):
    my_tree = et.parse(path)
    my_root = my_tree.getroot()
    
    temp_dict = {}

    curr_parent = None

    for node in my_root.iter():
        node_dict = node.attrib

        # Exception case for RebateBilling 
        if 'title' in node_dict and node_dict['title'] == 'Billing':
            title = node_dict['title']
            url = node_dict['url']
            temp_dict[title] = url.replace('~', 'C:/Projects/StockLoans/Src/SLTrader')
            curr_parent = title
            continue

        if 'location' in node_dict:
            title = node_dict['title']
            temp_dict[title] = {}
            curr_parent = title
        elif 'url' in node_dict:
            sub_title = node_dict['title']
            url = node_dict['url']
            temp_dict[curr_parent][sub_title] = url.replace('~', 'C:/Projects/StockLoans/Src/SLTrader')
            
    return temp_dict

In [4]:
# Checks if attributes t_1 to t_5 are included in the .cshtml file 

def deafult_column_settings_check(path):
    t_1 = '.Scrollable'
    t_2 = '.Reorderable'
    t_3 = '.Navigatable'
    t_4 = '.Selectable'
    t_5 = '.Resizable'
    
    grid_name_mark = '.Name('
    
    start_mark = 'Html.Kendo().Grid'
    end_mark = '.Read'
    end_mark_2 = '.Action'
    
    can_check = False
    
    is_scrollable = False
    is_reorderable = False
    is_navigatable = False
    is_selectable = False
    is_resizable = False
    
    error_msg = []
    
    current_grid = 0
    current_grid_name = ''
                    
    with open(path, 'r') as f2:
        lines = f2.readlines()
        for line in lines:
            # Marks checking start
            if start_mark.lower() in line.lower():
                can_check = True
                current_grid = current_grid + 1
                continue
            
            # Marks checking stop and append the error msg if any
            if (end_mark.lower() and end_mark_2.lower() in line.lower()) and can_check is True:
                can_check = False
                
                if is_scrollable is False:
                    error_msg.append('not Scrollable in grid ' + str(current_grid_name))
                elif is_reorderable is False:
                    error_msg.append('not Reorderable in grid ' + str(current_grid_name))
                elif is_navigatable is False:
                    error_msg.append('not Navigatable in grid ' + str(current_grid_name))
                elif is_selectable is False:
                    error_msg.append('not Selectable in grid ' + str(current_grid_name))
                elif is_resizable is False:
                    error_msg.append('not Resizable in grid ' + str(current_grid_name))
                
                # Resets check flags
                is_scrollable = False
                is_reorderable = False
                is_navigatable = False
                is_selectable = False
                is_resizable = False
                    
                continue
            
            # Check not allowed, continued...
            if can_check is False:
                continue
            
            # Extracts and stores the current grid name 
            if grid_name_mark.lower() in line.lower():
                my_str = line.strip()
                current_grid_name = my_str[7: len(my_str) - 2]
                continue
            
            # Checks
            if t_1 in line:
                is_scrollable = True
            elif t_2 in line:
                is_reorderable = True
            elif t_3 in line:
                is_navigatable = True
            elif t_4 in line:
                is_selectable = True
            elif t_5 in line:
                is_resizable = True
    
                
        return error_msg       

In [5]:
rebate_billing_path = 'c:/projects/stockloans/src/SLTrader/SiteMaps/Web-RebateBilling.sitemap'

rebate_billing_dict = {}

rebate_billing_dict = parse_xml_sitemap(rebate_billing_path)

In [6]:
check_report = {}

for i in rebate_billing_dict:
    if isinstance(rebate_billing_dict[i], dict):
        for j in rebate_billing_dict[i]:
            my_path = rebate_billing_dict[i][j]
            my_error_msg = deafult_column_settings_check(my_path)
            check_report[my_path] = my_error_msg
    elif isinstance(rebate_billing_dict[i], str):
        my_path = rebate_billing_dict[i]
        my_error_msg = deafult_column_settings_check(my_path)
        check_report[my_path] = my_error_msg
        
check_report

{'C:/Projects/StockLoans/Src/SLTrader/Areas/RebateBilling/Views/Groups/_AccountMaintenance.cshtml': ['not Reorderable in grid GroupAccountDetails'],
 'C:/Projects/StockLoans/Src/SLTrader/Areas/RebateBilling/Views/Groups/_IssueMaintenance.cshtml': ['not Reorderable in grid RebateBillingIssueOverrideDetails'],
 'C:/Projects/StockLoans/Src/SLTrader/Areas/RebateBilling/Views/RebateBilling/_RebateBilling.cshtml': ['not Reorderable in grid RebateBillingSummaryGrid',
  'not Reorderable in grid RebateBillingItemGrid'],
 'C:/Projects/StockLoans/Src/SLTrader/Areas/DomesticTrading/Views/FundingRates/_FeeTypes.cshtml': ['not Reorderable in grid FeeTypeGrid',
  'not Reorderable in grid FundingRateGrid'],
 'C:/Projects/StockLoans/Src/SLTrader/Areas/RebateBilling/Views/Research/_Contracts.cshtml': ['not Reorderable in grid ContractDetails'],
 'C:/Projects/StockLoans/Src/SLTrader/Areas/RebateBilling/Views/Research/_Fails.cshtml': ['not Reorderable in grid FailDetails'],
 'C:/Projects/StockLoans/Src/SL